In [1]:
import pandas as pd

## Peek at train

In [18]:
train = pd.read_csv('data_in/TrainingData.csv', na_values=['NO_LABEL'])
assert not train['Unnamed: 0'].duplicated().any()
train = train.set_index("Unnamed: 0")
train.shape

(400277, 25)

In [19]:
train.columns

Index(['Function', 'Use', 'Sharing', 'Reporting', 'Student_Type',
       'Position_Type', 'Object_Type', 'Pre_K', 'Operating_Status',
       'Object_Description', 'Text_2', 'SubFund_Description',
       'Job_Title_Description', 'Text_3', 'Text_4', 'Sub_Object_Description',
       'Location_Description', 'FTE', 'Function_Description',
       'Facility_or_Department', 'Position_Extra', 'Total',
       'Program_Description', 'Fund_Description', 'Text_1'],
      dtype='object')

In [20]:
train.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,...,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
134338,Teacher Compensation,Instruction,School Reported,School,NaN,Teacher,NaN,NaN,PreK-12 Operating,NaN,...,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Operating,CONTRACTOR SERVICES,...,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,...,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NaN,PreK-12 Operating,EMPLOYEE BENEFITS,...,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NaN,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,...,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


## Peek at test

In [21]:
test = pd.read_csv('data_in/TestData.csv', na_values=['NO_LABEL'])
assert not test['Unnamed: 0'].duplicated().any()
test = test.set_index("Unnamed: 0")
test.shape

/home/ubuntu/.local/share/virtualenvs/DRIVENDATA_SUBMISSIONS/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(50064, 16)

In [22]:
test.columns

Index(['Object_Description', 'Program_Description', 'SubFund_Description',
       'Job_Title_Description', 'Facility_or_Department',
       'Sub_Object_Description', 'Location_Description', 'FTE',
       'Function_Description', 'Position_Extra', 'Text_4', 'Total', 'Text_2',
       'Text_3', 'Fund_Description', 'Text_1'],
      dtype='object')

In [6]:
test.head()

,Object_Description,Program_Description,SubFund_Description,Job_Title_Description,Facility_or_Department,Sub_Object_Description,Location_Description,FTE,Function_Description,Position_Extra,Text_4,Total,Text_2,Text_3,Fund_Description,Text_1
Unnamed: 0,,,,,,,,,,,,,,,,
180042,Student Meals/Room/Other,Basic Educational Services,NaN,NaN,NaN,Line Item that is paid with Campus' money,School,NaN,Instruction,NaN,NaN,3999.910000,NaN,NaN,General Fund,NaN
28872,Extra Duty/Signing Bonus Pay,Undistributed,NaN,CHEERLEADER DIR,NaN,General,School,NaN,Cocurricular & Extra Curricular Activities,NaN,NaN,3447.320213,NaN,NaN,General Fund,NaN
186915,Professional Salaries,Bilingual Education,NaN,T-EL 1ST BIL,NaN,General,School,1.0,Instruction,NaN,NaN,52738.780869,NaN,NaN,General Fund,NaN
412396,Professional Salaries,Bilingual Education,NaN,T-EL 2ND BIL,NaN,General,School,1.0,Instruction,NaN,NaN,69729.263191,NaN,NaN,General Fund,NaN
427740,Salaries for Support Personnel,Undistributed,NaN,CLERK III- SCH,NaN,General,School,1.0,School Leadership,NaN,NaN,29492.834215,NaN,NaN,General Fund,NaN


## Diff columns

In [17]:
#train.loc[134338]
# pd.isnull(train['Function']).any()
# pd.isnull(test['Function']).any()
set(train.columns) - set(test.columns), set(test.columns) - set(train.columns)

({'Function',
  'Object_Type',
  'Operating_Status',
  'Position_Type',
  'Pre_K',
  'Reporting',
  'Sharing',
  'Student_Type',
  'Use'},
 set())

# Some guessing

Function = Teacher Compensation   =>  Use = Instruction

In [9]:
set(train['Use'][train['Function']=='Teacher Compensation'].values)

{'Instruction'}

Position Extra = TEACHER   =?=>  Function = Teacher Compensation

In [12]:
train['Function'][train['Position_Extra']=='TEACHER'].value_counts()

Teacher Compensation          3775
Professional Development        87
Aides Compensation              38
Physical Health & Services       8
NO_LABEL                         7
Student Transportation           5
Substitute Compensation          2
Extended Time & Tutoring         1
Social & Emotional               1
Name: Function, dtype: int64

In [16]:
train['Position_Extra'][train['Function']=='Teacher Compensation'].value_counts().head()

PROFESSIONAL-INSTRUCTIONAL    38621
UNDESIGNATED                  12130
TEACHER BACHELOR               6888
TEACHER                        3775
TIME CARD CERTIFIEDAddl        2571
Name: Position_Extra, dtype: int64

In [11]:
train['Function'][train['Position_Extra']=='TEACHER'].shape, train.shape

((3924,), (400277, 25))

In [13]:
test[test['Position_Extra']=='TEACHER'].shape, test.shape

((184, 16), (50064, 16))

## Read labels

In [28]:
import yaml
labels = yaml.load(open("labels.yml",'r'))

In [29]:
labels

{'Function': ['Aides Compensation',
  'Career & Academic Counseling',
  'Communications',
  'Curriculum Development',
  'Data Processing & Information Services',
  'Development & Fundraising',
  'Enrichment',
  'Extended Time & Tutoring',
  'Facilities & Maintenance',
  'Facilities Planning',
  'Finance, Budget, Purchasing & Distribution',
  'Food Services',
  'Governance',
  'Human Resources',
  'Instructional Materials & Supplies',
  'Insurance',
  'Legal',
  'Library & Media',
  'NO_LABEL',
  'Other Compensation',
  'Other Non-Compensation',
  'Parent & Community Relations',
  'Physical Health & Services',
  'Professional Development',
  'Recruitment',
  'Research & Accountability',
  'School Administration',
  'School Supervision',
  'Security & Safety',
  'Social & Emotional',
  'Special Population Program Management & Support',
  'Student Assignment',
  'Student Transportation',
  'Substitute Compensation',
  'Teacher Compensation',
  'Untracked Budget Set-Aside',
  'Utilities'],

## Calculate probabilities from training data

In [41]:
# Function': ['Aides Compensation
prediction_names = []
for k,v1 in labels.items():
    for v2 in v1:
        prediction_names.append("%s__%s"%(k,v2))
        
        
assert 'Function__Aides Compensation' in prediction_names
prediction_names[:5]

['Operating_Status__Non-Operating',
 'Operating_Status__Operating, Not PreK-12',
 'Operating_Status__PreK-12 Operating',
 'Pre_K__NO_LABEL',
 'Pre_K__Non PreK']

In [ ]:
train_prob = pd.DataFrame(columns=prediction_names)
train_prob.head()